In [580]:
import pandas as pd
import numpy as np
import re
import random
import pickle
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
from IPython.display import clear_output

### 1st Prototype

In [127]:
games=pd.read_csv('datasets/games.csv',index_col=0)
games_clusters=pd.read_csv('datasets/games_wt_clusters.csv',index_col=0)

In [139]:
# ge=pd.read_csv('datasets/genres.csv')
# pl=pd.read_csv('datasets/platforms.csv')

In [156]:
pl_cols=list(pickle.load(open('datasets/pl_cols.pkl', 'rb')))
ge_cols=list(pickle.load(open('datasets/ge_cols.pkl', 'rb')))

In [ ]:
scaler=pickle.load(open('models/scaler.sav','rb'))
model= pickle.load(open('models/cluster_model.sav','rb'))

In [4]:
games_clusters['title']

0                                        Elden Ring
1           The Legend of Zelda: Breath of the Wild
2                                             Hades
3                                     Hollow Knight
4                                         Undertale
                            ...                    
19923                                Planet Crafter
19924                               FIFA Manager 14
19925                                  Super Widget
19926                            Pizza Delivery Boy
19927    Pathfinder: Kingmaker - Definitive Edition
Name: title, Length: 19928, dtype: object

In [5]:
sorted_games= games_clusters.sort_values(by=['clusters','rating'],ascending=[True,False]).reset_index(drop=True)

In [6]:
sorted_games['title']=sorted_games['title'].str.lower()

In [7]:
x in sorted_games['title'].values

True

In [8]:
# if x in sorted_games['title'].values:
    

In [9]:
sorted_games['title'][sorted_games['title']=='batman']

16845    batman
Name: title, dtype: object

In [10]:
sorted_games.loc[16845]

title                                                      batman
release_date                                           1990-10-12
developers                                            ['Sunsoft']
summary         Batman is an Action game, developed and publis...
platforms                             ['turbografx-16/pc engine']
genres                                                ['brawler']
rating                                                        2.6
plays                                                          24
playing                                                         0
backlogs                                                       10
wishlist                                                        0
lists                                                           7
reviews                                                         4
clusters                                                       37
Name: 16845, dtype: object

In [11]:
sorted_games.shape

(19928, 14)

In [219]:
sorted_games.head()

,title,release_date,developers,summary,platforms,genres,rating,plays,playing,backlogs,wishlist,lists,reviews,clusters
0,half-life: alyx,2020-03-23,['Valve'],Half-Life: Alyx is Valve’s VR return to the Ha...,"['steamvr', 'windows mixed reality', 'oculus r...","['adventure', 'puzzle', 'shooter']",4.5,2200,247,1400,1600,563,221,0
1,walkabout mini golf,2020-09-24,['Mighty Coconut'],Putt your way through beautifully designed cou...,"['oculus vr', 'steamvr', 'oculus quest', 'ocul...","['indie', 'sport']",4.4,85,19,19,20,27,8,0
2,walkabout mini golf: myst,2022-11-15,['Mighty Coconut'],"An upcoming DLC for Walkabout Mini Golf, featu...","['steamvr', 'windows mixed reality', 'oculus r...","['indie', 'sport']",4.4,4,0,3,2,0,0,0
3,the last clockwinder,2022-06-02,"['Cyan Ventures', 'Pontoco']",Create intricate clockwork contraptions out of...,"['steamvr', 'oculus quest 2', 'oculus rift', '...","['adventure', 'indie', 'puzzle', 'simulator']",4.3,28,7,33,52,25,7,0
4,moss: book ii,2022-03-31,['Polyarc Games'],Quill is back! And she's journeying deep into ...,"['steamvr', 'playstation vr', 'oculus quest 2'...","['adventure', 'platform', 'puzzle']",4.3,28,7,47,65,27,4,0


### Scraping

In [13]:
from bs4 import BeautifulSoup
import requests

In [704]:
def start():
        x=input('Enter a game title: ')
        if x.lower()!='done':
            get_info(x)
        else:
            clear_output()
            #add recommendation here so it doesnt clear out
            print('Hope you enjoy the recommendation!')

In [709]:
def get_info(game):
    url=f'https://www.backloggd.com/search/games/{game}'
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"html.parser")
    
    search=pd.DataFrame(columns=['Name'])
    
    for n in soup.select('div > a > h3'):
        name = n.get_text().strip()
        search=pd.concat([search,pd.DataFrame({'Name':[name]})],ignore_index=True)

    display(search)
    
    print('If you dont like what you see, type "-1"', '\n')
    b= input('Enter the number you see your title on: ')
    if b=='-1':
        clear_output()
        print('Search again- ')
        start()
    else:
        try:
            b=int(b)
            m=search['Name'][b].lower()
            m=re.sub("'",'',m)
            m=re.sub(r'[^a-zA-Z0-9]+', '-', m)
            final_df=get_game_data(m)
        except (ValueError, IndexError):
            print('Invalid input!', '\n')
            print('Starting again...')
            start()
            
    pred=model.predict(final_df)[0]
    print(pred)

In [710]:
def get_game_data(m):
    url2=f"https://www.backloggd.com/games/{m}/"
    response=requests.get(url2)
    soup=BeautifulSoup(response.content,"html.parser")
    
    # getting categorical data: platforms and genres
    platforms=pd.DataFrame(0,index=[0],columns=pl_cols)
    genres= pd.DataFrame(0,index=[0],columns=ge_cols)
    
    pf=[]   
    for  p in soup.select('div > a.game-page-platform '):
        pf.append(p.get_text().strip().lower().replace(' ','_'))
    
    for i in pf:
        if i in platforms.columns:
            platforms[i]=1
            
    ge=[]
    for g in soup.select('div > p.genre-tag > a'):
        ge.append(g.get_text().strip().lower().replace(' ','_'))
    
    for g in ge:
        if g in genres.columns:
            genres[g]=1
    
    #getting numerical data
    num_data=pd.DataFrame(0,index=[0],columns=['rating','plays','playing','backlogs','wishlist','lists','reviews'])
    
    num_data['rating']= float(soup.select('#score > h1')[0].get_text().strip())
    
    for index, column in enumerate(num_data.columns[1:5]):
        num_data[column]=soup.select('div.col-auto.ml-auto.pl-0 > p')[index].get_text().strip()
        num_data[column]=num_data[column].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
    
    for index, column in enumerate(num_data.columns[5:]):
        num_data[column]=soup.select('div > a > p.game-page-sidecard')[index].get_text().strip()
        num_data[column]=num_data[column].str.split(' ').str[0]
        num_data[column]=num_data[column].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)
        
    #scaling the numerical data
    num_trans=pd.DataFrame(scaler.transform(num_data),columns= num_data.columns)
    
    final_data=pd.concat([num_trans,platforms,genres],axis=1)
    
    return final_data

In [712]:
start()

Enter a game title: baldur


,Name
0,Baldur's Gate 3
1,Baldur's Gate
2,Baldur's Gate II: Shadows of Amn
3,Baldur's Gate: Enhanced Edition
4,Baldur's Gate: Dark Alliance
5,Baldur's Gate II: Enhanced Edition
6,Baldur's Gate: Dark Alliance II
7,Baldur's Gate II: Throne of Bhaal
8,Baldur's Gate: Tales of the Sword Coast
9,Baldur's Gate: Siege of Dragonspear


If you dont like what you see, type "-1" 

Enter the number you see your title on: 6
28


In [ ]:
# in the final function if you would like to search again or not

In [ ]:
get_info(x)

In [635]:
x=input('Enter a game title: ')

Enter a game title: batman


In [636]:
url=f'https://www.backloggd.com/search/games/{x}'

response=requests.get(url)

In [637]:
response

<Response [200]>

In [638]:
soup=BeautifulSoup(response.content,"html.parser")

In [639]:
#soup

In [640]:
#my-posts > div:nth-child(1) > div > div.col.px-0 > div > div > div > div.col-auto.px-0.game-name > a > h3

In [641]:
#soup.select('#my-posts > div:nth-child(n) > div > div > div > div > div > div > a > h3')

In [642]:
# name=[]

# for n in soup.select('#my-posts > div:nth-child(n) > div > div > div > div > div > div > a > h3'):
#     name.append(n.get_text().strip())

In [643]:
search=pd.DataFrame(columns=['Name'])

for n in soup.select('div > a > h3'):
    name = n.get_text().strip()
    search=pd.concat([search,pd.DataFrame({'Name':[name]})],ignore_index=True)

In [644]:
search

,Name
0,Batman: Arkham Knight
1,Batman: Arkham Origins
2,Batman: Return to Arkham - Arkham Asylum
3,The Adventures of Batman & Robin
4,Batman: Vengeance
5,Batman: Arkham Knight - Red Hood Story Pack
6,Batman: Arkham VR
7,Batman Forever
8,"Batman: Arkham Origins - Cold, Cold Heart"
9,Batman: The Video Game


In [213]:
# c=input('Is the game in this list?  0 : No, 1 : Yes - ')

Is the game in this list?  0 : No, 1 : Yes - 1


In [ ]:
# if c==0:
#     x= input('Please enter title differently: ')
#     url= 

In [645]:
b= int(input('Enter the number you see your title on: '))

Enter the number you see your title on: 1


In [646]:

m=search['Name'][b].lower()
m=re.sub("'",'',m)
m=re.sub(r'[^a-zA-Z0-9]+', '-', m)

In [647]:
m

'batman-arkham-origins'

In [648]:
url2=f"https://www.backloggd.com/games/{m}/"
#use this to scrape info about the game from the site

In [649]:
url2

'https://www.backloggd.com/games/batman-arkham-origins/'

In [650]:
response=requests.get(url2)
response

<Response [200]>

In [651]:
soup=BeautifulSoup(response.content,"html.parser")

In [652]:
#soup

### getting categorical data

In [653]:
#scraped_game= pd.DataFrame(columns=['platforms','genres','rating','plays','playing','backlogs','wishlist','lists','reviews'])

In [654]:
platforms=pd.DataFrame(0,index=[0],columns=pl_cols)

In [655]:
platforms

,1292_advanced_programmable_video_system,3do_interactive_multiplayer,acorn_archimedes,acorn_electron,amazon_fire_tv,amiga,amiga_cd32,amstrad_cpc,amstrad_pcw,android,apple_ii,apple_iigs,arcade,arduboy,atari_2600,atari_5200,atari_7800,atari_8-bit,atari_jaguar,atari_jaguar_cd,atari_lynx,atari_st/ste,ay-3-8606,bally_astrocade,bbc_microcomputer_system,blackberry_os,blu-ray_player,cdc_cyber_70,colecovision,commodore_16,commodore_c64/128,commodore_cdtv,commodore_pet,commodore_plus/4,commodore_vic-20,daydream,dragon_32/64,dreamcast,dvd_player,edsac,fairchild_channel_f,family_computer_(famicom),family_computer_disk_system,fm_towns,fm-7,gamate,game_&_watch,game_boy,game_boy_advance,game_boy_color,game.com,gear_vr,google_stadia,handheld_electronic_lcd,hp_2100,hyper_neo_geo_64,intellivision,intellivision_amico,ios,leapster,leapster_explorer/leadpad_explorer,legacy_computer,linux,mac,microcomputer,mobile,msx,msx2,n-gage,nec_pc-6000_series,neo_geo_aes,neo_geo_cd,neo_geo_mvs,neo_geo_pocket,neo_geo_pocket_color,nes,new_nintendo_3ds,nintendo_3ds,nintendo_64,nintendo_64dd,nintendo_ds,nintendo_dsi,nintendo_gamecube,nintendo_playstation,nintendo_switch,oculus_quest,oculus_quest_2,oculus_rift,oculus_vr,odyssey,onlive_game_system,ooparts,ouya,palm_os,pc_dos,pc_engine_supergrafx,pc-8801,pc-98,pc-fx,pdp-10,philips_cd-i,philips_videopac_g7000,playdate,playstation,playstation_2,playstation_3,playstation_4,playstation_5,playstation_network,playstation_portable,playstation_vita,playstation_vr,playstation_vr2,plug_&_play,pokémon_mini,satellaview,sega_32x,sega_cd,sega_game_gear,sega_master_system,sega_mega_drive/genesis,sega_saturn,sg-1000,sharp_mz-2200,sharp_x1,sharp_x68000,snes,stadia,steamvr,super_famicom,tapwave_zodiac,tatung_einstein,texas_instruments_ti-99,thomson_mo5,trs-80,trs-80_color_computer,turbografx-16/pc_engine,turbografx-16/pc_engine_cd,v.smile,vc_4000,vectrex,virtual_boy,web_browser,wii,wii_u,windows_mixed_reality,windows_mobile,windows_pc,windows_phone,wonderswan,wonderswan_color,xbox,xbox_360,xbox_one,xbox_series,zeebo,zx_spectrum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [656]:
pf=[]
for  p in soup.select('div > a.game-page-platform '):
    pf.append(p.get_text().strip().lower().replace(' ','_'))
    
pf

['windows_pc', 'wii_u', 'xbox_360', 'ios', 'playstation_3']

In [657]:
for i in pf:
    if i in platforms.columns:
        platforms[i]=1

In [658]:
platforms

,1292_advanced_programmable_video_system,3do_interactive_multiplayer,acorn_archimedes,acorn_electron,amazon_fire_tv,amiga,amiga_cd32,amstrad_cpc,amstrad_pcw,android,apple_ii,apple_iigs,arcade,arduboy,atari_2600,atari_5200,atari_7800,atari_8-bit,atari_jaguar,atari_jaguar_cd,atari_lynx,atari_st/ste,ay-3-8606,bally_astrocade,bbc_microcomputer_system,blackberry_os,blu-ray_player,cdc_cyber_70,colecovision,commodore_16,commodore_c64/128,commodore_cdtv,commodore_pet,commodore_plus/4,commodore_vic-20,daydream,dragon_32/64,dreamcast,dvd_player,edsac,fairchild_channel_f,family_computer_(famicom),family_computer_disk_system,fm_towns,fm-7,gamate,game_&_watch,game_boy,game_boy_advance,game_boy_color,game.com,gear_vr,google_stadia,handheld_electronic_lcd,hp_2100,hyper_neo_geo_64,intellivision,intellivision_amico,ios,leapster,leapster_explorer/leadpad_explorer,legacy_computer,linux,mac,microcomputer,mobile,msx,msx2,n-gage,nec_pc-6000_series,neo_geo_aes,neo_geo_cd,neo_geo_mvs,neo_geo_pocket,neo_geo_pocket_color,nes,new_nintendo_3ds,nintendo_3ds,nintendo_64,nintendo_64dd,nintendo_ds,nintendo_dsi,nintendo_gamecube,nintendo_playstation,nintendo_switch,oculus_quest,oculus_quest_2,oculus_rift,oculus_vr,odyssey,onlive_game_system,ooparts,ouya,palm_os,pc_dos,pc_engine_supergrafx,pc-8801,pc-98,pc-fx,pdp-10,philips_cd-i,philips_videopac_g7000,playdate,playstation,playstation_2,playstation_3,playstation_4,playstation_5,playstation_network,playstation_portable,playstation_vita,playstation_vr,playstation_vr2,plug_&_play,pokémon_mini,satellaview,sega_32x,sega_cd,sega_game_gear,sega_master_system,sega_mega_drive/genesis,sega_saturn,sg-1000,sharp_mz-2200,sharp_x1,sharp_x68000,snes,stadia,steamvr,super_famicom,tapwave_zodiac,tatung_einstein,texas_instruments_ti-99,thomson_mo5,trs-80,trs-80_color_computer,turbografx-16/pc_engine,turbografx-16/pc_engine_cd,v.smile,vc_4000,vectrex,virtual_boy,web_browser,wii,wii_u,windows_mixed_reality,windows_mobile,windows_pc,windows_phone,wonderswan,wonderswan_color,xbox,xbox_360,xbox_one,xbox_series,zeebo,zx_spectrum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0


In [659]:
#soup.select('#game-body > div> div:nth-child(n) > div> div:nth-child(n) > div > a')

In [660]:
#game-body > div.col > div:nth-child(2) > div.col-lg-4.mt-1.mt-lg-2.col-12 > div:nth-child(4) > div > p > a

In [661]:
genres= pd.DataFrame(0,index=[0],columns=ge_cols)

In [662]:
ge=[]
for g in soup.select('div > p.genre-tag > a'):
    ge.append(g.get_text().strip().lower().replace(' ','_'))
ge

['adventure', 'brawler']

In [663]:
for g in ge:
    if g in genres.columns:
        genres[g]=1

In [664]:
genres

,adventure,arcade,brawler,card_&_board_game,fighting,indie,moba,music,pinball,platform,point-and-click,puzzle,quiz/trivia,racing,real_time_strategy,rpg,shooter,simulator,sport,strategy,tactical,turn_based_strategy,visual_novel
0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### getting numeric data

In [665]:
num_data=pd.DataFrame(0,index=[0],columns=['rating','plays','playing','backlogs','wishlist','lists','reviews'])

In [666]:
num_data['rating']= float(soup.select('#score > h1')[0].get_text().strip())

In [667]:
for index, column in enumerate(num_data.columns[1:5]):
    num_data[column]=soup.select('div.col-auto.ml-auto.pl-0 > p')[index].get_text().strip()
    num_data[column]=num_data[column].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)

In [668]:
for index, column in enumerate(num_data.columns[5:]):
    num_data[column]=soup.select('div > a > p.game-page-sidecard')[index].get_text().strip()
    num_data[column]=num_data[column].str.split(' ').str[0]
    num_data[column]=num_data[column].replace({"K":"*1e3"}, regex=True).map(pd.eval).astype(int)

In [669]:
num_data

,rating,plays,playing,backlogs,wishlist,lists,reviews
0,3.4,11000,181,2700,917,946,472


In [670]:
num_data.dtypes

rating      float64
plays         int32
playing       int32
backlogs      int32
wishlist      int32
lists         int32
reviews       int32
dtype: object

#### scaling the numerical

In [507]:
scaler=pickle.load(open('models/scaler.sav','rb'))

In [508]:
num_trans=pd.DataFrame(scaler.transform(num_data),columns= num_data.columns)

In [509]:
num_trans

,rating,plays,playing,backlogs,wishlist,lists,reviews
0,0.574468,0.001368,0.000217,0.000208,0.000364,0.00098,0.003


### Final Data

In [510]:
final_data=pd.concat([num_trans,platforms,genres],axis=1)

In [511]:
final_data

,rating,plays,playing,backlogs,wishlist,lists,reviews,1292_advanced_programmable_video_system,3do_interactive_multiplayer,acorn_archimedes,acorn_electron,amazon_fire_tv,amiga,amiga_cd32,amstrad_cpc,amstrad_pcw,android,apple_ii,apple_iigs,arcade,arduboy,atari_2600,atari_5200,atari_7800,atari_8-bit,atari_jaguar,atari_jaguar_cd,atari_lynx,atari_st/ste,ay-3-8606,bally_astrocade,bbc_microcomputer_system,blackberry_os,blu-ray_player,cdc_cyber_70,colecovision,commodore_16,commodore_c64/128,commodore_cdtv,commodore_pet,commodore_plus/4,commodore_vic-20,daydream,dragon_32/64,dreamcast,dvd_player,edsac,fairchild_channel_f,family_computer_(famicom),family_computer_disk_system,fm_towns,fm-7,gamate,game_&_watch,game_boy,game_boy_advance,game_boy_color,game.com,gear_vr,google_stadia,handheld_electronic_lcd,hp_2100,hyper_neo_geo_64,intellivision,intellivision_amico,ios,leapster,leapster_explorer/leadpad_explorer,legacy_computer,linux,mac,microcomputer,mobile,msx,msx2,n-gage,nec_pc-6000_series,neo_geo_aes,neo_geo_cd,neo_geo_mvs,neo_geo_pocket,neo_geo_pocket_color,nes,new_nintendo_3ds,nintendo_3ds,nintendo_64,nintendo_64dd,nintendo_ds,nintendo_dsi,nintendo_gamecube,nintendo_playstation,nintendo_switch,oculus_quest,oculus_quest_2,oculus_rift,oculus_vr,odyssey,onlive_game_system,ooparts,ouya,palm_os,pc_dos,pc_engine_supergrafx,pc-8801,pc-98,pc-fx,pdp-10,philips_cd-i,philips_videopac_g7000,playdate,playstation,playstation_2,playstation_3,playstation_4,playstation_5,playstation_network,playstation_portable,playstation_vita,playstation_vr,playstation_vr2,plug_&_play,pokémon_mini,satellaview,sega_32x,sega_cd,sega_game_gear,sega_master_system,sega_mega_drive/genesis,sega_saturn,sg-1000,sharp_mz-2200,sharp_x1,sharp_x68000,snes,stadia,steamvr,super_famicom,tapwave_zodiac,tatung_einstein,texas_instruments_ti-99,thomson_mo5,trs-80,trs-80_color_computer,turbografx-16/pc_engine,turbografx-16/pc_engine_cd,v.smile,vc_4000,vectrex,virtual_boy,web_browser,wii,wii_u,windows_mixed_reality,windows_mobile,windows_pc,windows_phone,wonderswan,wonderswan_color,xbox,xbox_360,xbox_one,xbox_series,zeebo,zx_spectrum,adventure,arcade,brawler,card_&_board_game,fighting,indie,moba,music,pinball,platform,point-and-click,puzzle,quiz/trivia,racing,real_time_strategy,rpg,shooter,simulator,sport,strategy,tactical,turn_based_strategy,visual_novel
0,0.574468,0.001368,0.000217,0.000208,0.000364,0.00098,0.003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Cluster prediction

In [512]:

pred=model.predict(final_data)

In [513]:
sorted_games[sorted_games['clusters']==pred[0]]

,title,release_date,developers,summary,platforms,genres,rating,plays,playing,backlogs,wishlist,lists,reviews,clusters
7253,morbius heardle,2022-05-31,[],"It is a clone of Sonic Heardle, a clone of Joy...",['web browser'],"['music', 'quiz/trivia']",5.0,10,0,0,2,2,2,16
7254,popsauce,2014-06-27,['Sparklin Labs'],Blind test for your eyes: challenge your frien...,['web browser'],"['arcade', 'indie', 'quiz/trivia']",4.8,6,0,2,0,3,2,16
7255,dominion online,2017-12-31,['Shuffle iT'],Dominion is the original deck-building card ga...,['web browser'],['card & board game'],4.4,16,1,0,1,3,1,16
7256,small worlds,2009-12-31,[],A short atmospheric game about exploring.,['web browser'],['platform'],4.3,10,0,5,3,5,1,16
7257,clown café,2017-12-31,[],Clown Café is a parody visual novel on Tumblr.,['web browser'],['visual novel'],4.3,5,0,1,1,2,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7543,farmville,2009-06-19,['Zynga'],FarmVille is a farming simulation social netwo...,['web browser'],"['simulator', 'strategy']",1.6,389,0,7,3,26,10,16
7544,breakout poetry,2020-05-09,['Johan Holm'],Break blocks. Free words. Create poems. A new ...,['web browser'],"['arcade', 'indie']",1.6,6,0,14,3,8,0,16
7545,tender frog house,2020-05-06,['gewl'],A wholesome one-day prototype.,['web browser'],"['fighting', 'indie', 'visual novel']",1.4,36,0,5,5,11,18,16
7546,epic fantasy quest game with extensive lore an...,2015-03-29,['Michael Lutz'],Epic Fantasy Quest Game with Extensive Lore an...,['web browser'],['indie'],1.0,8,0,2,3,3,0,16


In [514]:
sorted_games[sorted_games['clusters']==pred[0]]

,title,release_date,developers,summary,platforms,genres,rating,plays,playing,backlogs,wishlist,lists,reviews,clusters
7253,morbius heardle,2022-05-31,[],"It is a clone of Sonic Heardle, a clone of Joy...",['web browser'],"['music', 'quiz/trivia']",5.0,10,0,0,2,2,2,16
7254,popsauce,2014-06-27,['Sparklin Labs'],Blind test for your eyes: challenge your frien...,['web browser'],"['arcade', 'indie', 'quiz/trivia']",4.8,6,0,2,0,3,2,16
7255,dominion online,2017-12-31,['Shuffle iT'],Dominion is the original deck-building card ga...,['web browser'],['card & board game'],4.4,16,1,0,1,3,1,16
7256,small worlds,2009-12-31,[],A short atmospheric game about exploring.,['web browser'],['platform'],4.3,10,0,5,3,5,1,16
7257,clown café,2017-12-31,[],Clown Café is a parody visual novel on Tumblr.,['web browser'],['visual novel'],4.3,5,0,1,1,2,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7543,farmville,2009-06-19,['Zynga'],FarmVille is a farming simulation social netwo...,['web browser'],"['simulator', 'strategy']",1.6,389,0,7,3,26,10,16
7544,breakout poetry,2020-05-09,['Johan Holm'],Break blocks. Free words. Create poems. A new ...,['web browser'],"['arcade', 'indie']",1.6,6,0,14,3,8,0,16
7545,tender frog house,2020-05-06,['gewl'],A wholesome one-day prototype.,['web browser'],"['fighting', 'indie', 'visual novel']",1.4,36,0,5,5,11,18,16
7546,epic fantasy quest game with extensive lore an...,2015-03-29,['Michael Lutz'],Epic Fantasy Quest Game with Extensive Lore an...,['web browser'],['indie'],1.0,8,0,2,3,3,0,16
